In [4]:
# !pip install tensorflow opencv-python

In [55]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np
import keras

vocab_size = 88584
maxlen = 250
batch_size = 64


(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = vocab_size)

In [56]:
train_data[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

# More Processing

1. If the review is greater than 250  words then trim off the extra words
2. if the review is less than 250 words add the necessary amount  of 's to make it equal to 250

In [57]:
train_data = sequence.pad_sequences(train_data, maxlen )
test_data = sequence.pad_sequences(test_data, maxlen)
train_data[1]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     1,   194,
        1153,   194,  8255,    78,   228,     5,     6,  1463,  4369,
        5012,   134,    26,     4,   715,     8,   118,  1634,    14,
         394,    20,    13,   119,   954,   189,   102,     5,   207,
         110,  3103,    21,    14,    69,   188,     8,    30,    23,
           7,     4,   249,   126,    93,     4,   114,     9,  2300,
        1523,     5,   647,     4,   116,     9,    35,  8163,     4,
         229,     9,   340,  1322,     4,   118,     9,     4,   130,
        4901,    19,

# Creating the model

Now it's time to create the model. we'll use a word embedding layer as the first layer in our model  and add LSTM layer afterwards that feeds into a dense node to get our predicted sentiment.


In [58]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.LSTM(32),  #32 means the output dimension of the generated vector
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 32)          2834688   
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Training model

Now it's time to compile and train the model
use loss function = binary_crossentropy

In [59]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(train_data, train_labels, epochs = 2, validation_split = 0.2)

Epoch 1/2


C:\Users\YASHASVI\Desktop\data science\env\Lib\site-packages\keras\src\backend.py:5820: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


625/625 [==============================] - 58s 83ms/step - loss: 0.4487 - accuracy: 0.7826 - val_loss: 0.2940 - val_accuracy: 0.8828
Epoch 2/2
625/625 [==============================] - 46s 73ms/step - loss: 0.2618 - accuracy: 0.8989 - val_loss: 0.2904 - val_accuracy: 0.8780


In [60]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 18s 23ms/step - loss: 0.2978 - accuracy: 0.8770
[0.29783228039741516, 0.8769599795341492]


# Make prediction

In [61]:
word_index = imdb.get_word_index()  # for getting the index of every word in dataset

In [62]:
def encode_text(text):
    tokens = keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    
    return sequence.pad_sequences([tokens], maxlen)[0]

text = 'that movie was just amazing , so amazing' 

encode = encode_text(text)
print(encode)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [63]:
# while were at it lets make a decode function  

reverse_word_index = {value : key for (key, value) in word_index.items()}

def decode_integers(integers):
    pad = 0
    text = ""
    
    for num in integers:
        if num != pad:
            text += reverse_word_index[num] + " "
     
    return text[:-1]
print(decode_integers(encode))    #reversing the integer back to the word or lines

that movie was just amazing so amazing


# Now lets take review to as input and predict it's review

In [78]:
# Now time to make prediction

def predict():
    encoded_text = encode_text(input("How much did you like this movvie : "))
    
    pred = np.zeros((1, 250))  #our model accept this shape 
    
    pred[0] = encoded_text     # now pred ka (x, y) x me enocde_text save kiyaa
    
    result = model.predict(pred)
    prob = result[0][0]
    return prob

# positive_review = 'That movie was so awesome ! I really loved it and would watch it again because it was amazingly great'

prob = predict()

# # negative_review = "That movie sucked . I hated it and wouldn't watch it again , was one off the worst ever i watched"

# # predict(negative_review)

# th = 0.5
if prob>0.5:
    class_ = 1
    review = 'Good'
elif prob<0.5:
    class_ = 0
    review = 'Not so good'
print(f"Probability and Predicted class of this Statement is {prob} and {class_}.")
print(f"Review of this movie is {review}")

How much did you like this movvie : bhaut hi ghatiya movie thi bhai mat dekhna kabhi bhi bakwas nikli ek number ki
1/1 [==============================] - 0s 59ms/step
Probability and Predicted class of this Statement is 0.4441061019897461 and 0.
Review of this movie is Not so good


# Lower the number of prediction more negative prediction it is and vice versa

# Now what if we get so many review at a time we need to predict their class

In [101]:
texts = []
for i in range(2):
    texts.append(input("Enter the responses on this movie"))


# Now time to make prediction

def predict():
    for text in texts:
        
        encoded_text = encode_text(text)

        pred = np.zeros((1, 250))  #our model accept this shape 

        pred[0] = encoded_text     # now pred ka (x, y) x me enocde_text save kiyaa

        result = model.predict(pred)
        prob = result[0][0]    
    return prob

# positive_review = 'That movie was so awesome ! I really loved it and would watch it again because it was amazingly great'

prob = predict()

# # negative_review = "That movie sucked . I hated it and wouldn't watch it again , was one off the worst ever i watched"

# # predict(negative_review)

# th = 0.5
if prob>0.5:
    class_ = 1
    review = 'Good'
elif prob<0.5:
    class_ = 0
    review = 'Not so good'
print(f"Probability and Predicted class of this Statement is {prob} and {class_}.")
print(f"Review of this movie is {review}")
    

Enter the responses on this moviebhaut hi ghatiya movie thi bhai mat dekhna kabhi bhi bakwas nikli ek number ki
Enter the responses on this movievery nice movie evry one should watch it it's wonnderful
1/1 [==============================] - 0s 41ms/step
Probability and Predicted class of this Statement is 0.5090075135231018 and 1.
Review of this movie is Good
